# Введение

В этом задании Вы продолжите работать с данными из семинара [Articles Sharing and Reading from CI&T Deskdrop](https://www.kaggle.com/gspmoreira/articles-sharing-reading-from-cit-deskdrop).

In [ ]:
import pandas as pd
import numpy as np
import math

## Загрузка и предобработка данных

Загрузим данные и проведем предобраотку данных как на семинаре.

In [ ]:
!wget -q -N https://www.dropbox.com/s/z8syrl5trawxs0n/articles.zip?dl=0 -O articles.zip
!unzip -o -q articles.zip

In [ ]:
articles_df = pd.read_csv('articles/shared_articles.csv')
articles_df = articles_df[articles_df['eventType'] == 'CONTENT SHARED']
articles_df.head(2)

,timestamp,eventType,contentId,authorPersonId,authorSessionId,authorUserAgent,authorRegion,authorCountry,contentType,url,title,text,lang
1,1459193988,CONTENT SHARED,-4110354420726924665,4340306774493623681,8940341205206233829,NaN,NaN,NaN,HTML,http://www.nytimes.com/2016/03/28/business/dea...,"Ethereum, a Virtual Currency, Enables Transact...",All of this work is still very early. The firs...,en
2,1459194146,CONTENT SHARED,-7292285110016212249,4340306774493623681,8940341205206233829,NaN,NaN,NaN,HTML,http://cointelegraph.com/news/bitcoin-future-w...,Bitcoin Future: When GBPcoin of Branson Wins O...,The alarm clock wakes me at 8:00 with stream o...,en


In [ ]:
interactions_df = pd.read_csv('articles/users_interactions.csv')
interactions_df.head(2)

,timestamp,eventType,contentId,personId,sessionId,userAgent,userRegion,userCountry
0,1465413032,VIEW,-3499919498720038879,-8845298781299428018,1264196770339959068,NaN,NaN,NaN
1,1465412560,VIEW,8890720798209849691,-1032019229384696495,3621737643587579081,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_2...,NY,US


In [ ]:
interactions_df.personId = interactions_df.personId.astype(str)
interactions_df.contentId = interactions_df.contentId.astype(str)
articles_df.contentId = articles_df.contentId.astype(str)

In [ ]:
# зададим словарь определяющий силу взаимодействия
event_type_strength = {
   'VIEW': 1.0,
   'LIKE': 2.0, 
   'BOOKMARK': 2.5, 
   'FOLLOW': 3.0,
   'COMMENT CREATED': 4.0,  
}

interactions_df['eventStrength'] = interactions_df.eventType.apply(lambda x: event_type_strength[x])

Оставляем только тех пользователей, которые произамодействовали более чем с пятью статьями.

In [ ]:
users_interactions_count_df = (
    interactions_df
    .groupby(['personId', 'contentId'])
    .first()
    .reset_index()
    .groupby('personId').size())
print('# users:', len(users_interactions_count_df))

users_with_enough_interactions_df = \
    users_interactions_count_df[users_interactions_count_df >= 5].reset_index()[['personId']]
print('# users with at least 5 interactions:',len(users_with_enough_interactions_df))

# users: 1895
# users with at least 5 interactions: 1140


Оставляем только те взаимодействия, которые относятся к отфильтрованным пользователям.

In [ ]:
interactions_from_selected_users_df = interactions_df.loc[np.in1d(interactions_df.personId,
            users_with_enough_interactions_df)]

In [ ]:
print('# interactions before:', interactions_df.shape)
print('# interactions after:', interactions_from_selected_users_df.shape)

# interactions before: (72312, 9)
# interactions after: (69868, 9)


Объединяем все взаимодействия пользователя по каждой статье и сглажиываем полученный результат, взяв от него логарифм.

In [ ]:
def smooth_user_preference(x):
    return math.log(1+x, 2)
    
interactions_full_df = (
    interactions_from_selected_users_df
    .groupby(['personId', 'contentId']).eventStrength.sum()
    .apply(smooth_user_preference)
    .reset_index().set_index(['personId', 'contentId'])
)
interactions_full_df['last_timestamp'] = (
    interactions_from_selected_users_df
    .groupby(['personId', 'contentId'])['timestamp'].last()
)
        
interactions_full_df = interactions_full_df.reset_index()
interactions_full_df.head(5)

,personId,contentId,eventStrength,last_timestamp
0,-1007001694607905623,-5065077552540450930,1.000000,1470395911
1,-1007001694607905623,-6623581327558800021,1.000000,1487240080
2,-1007001694607905623,-793729620925729327,1.000000,1472834892
3,-1007001694607905623,1469580151036142903,1.000000,1487240062
4,-1007001694607905623,7270966256391553686,1.584963,1485994324


Разобьём выборку на обучение и контроль по времени.

In [ ]:
from sklearn.model_selection import train_test_split

split_ts = 1475519530
interactions_train_df = interactions_full_df.loc[interactions_full_df.last_timestamp < split_ts].copy()
interactions_test_df = interactions_full_df.loc[interactions_full_df.last_timestamp >= split_ts].copy()

print('# interactions on Train set: %d' % len(interactions_train_df))
print('# interactions on Test set: %d' % len(interactions_test_df))

interactions_train_df

# interactions on Train set: 29329
# interactions on Test set: 9777


,personId,contentId,eventStrength,last_timestamp
0,-1007001694607905623,-5065077552540450930,1.0,1470395911
2,-1007001694607905623,-793729620925729327,1.0,1472834892
6,-1032019229384696495,-1006791494035379303,1.0,1469129122
7,-1032019229384696495,-1039912738963181810,1.0,1459376415
8,-1032019229384696495,-1081723567492738167,2.0,1464054093
...,...,...,...,...
39099,997469202936578234,9112765177685685246,2.0,1472479493
39100,998688566268269815,-1255189867397298842,1.0,1474567164
39101,998688566268269815,-401664538366009049,1.0,1474567449
39103,998688566268269815,6881796783400625893,1.0,1474567675


Для удобства подсчёта качества запишем данные в формате, где строка соответствует пользователю, а столбцы будут истинными метками и предсказаниями в виде списков.

In [ ]:
interactions = (
    interactions_train_df
    .groupby('personId')['contentId'].agg(lambda x: list(x))
    .reset_index()
    .rename(columns={'contentId': 'true_train'})
    .set_index('personId')
)

interactions['true_test'] = (
    interactions_test_df
    .groupby('personId')['contentId'].agg(lambda x: list(x))
)

# заполнение пропусков пустыми списками
interactions.loc[pd.isnull(interactions.true_test), 'true_test'] = [
    list() for x in range(len(interactions.loc[pd.isnull(interactions.true_test), 'true_test']))]

interactions.head(1)

,true_train,true_test
personId,,
-1007001694607905623,"[-5065077552540450930, -793729620925729327]","[-6623581327558800021, 1469580151036142903, 72..."


## Библиотека LightFM

Для рекомендации Вы будете пользоваться библиотекой [LightFM](https://making.lyst.com/lightfm/docs/home.html), в которой реализованы популярные алгоритмы. Для оценивания качества рекомендации, как и на семинаре, будем пользоваться метрикой *precision@10*.

In [ ]:
!pip install lightfm
from lightfm import LightFM
from lightfm.evaluation import precision_at_k

     |████████████████████████████████| 317kB 8.5MB/s 
  Created wheel for lightfm: filename=lightfm-1.16-cp37-cp37m-linux_x86_64.whl size=705331 sha256=05214a0a2b0db31b64a92c788daba023b1ad669ab23e9f24b17942c568f28c4f
  Stored in directory: /root/.cache/pip/wheels/c6/64/d4/673c7277f71ac4c5ad4835b94708c01b653ef2d3aa78ef20aa
Successfully built lightfm


## Задание 1. (2 балла)

Модели в LightFM работают с разреженными матрицами. Создайте разреженные матрицы `data_train` и `data_test` (размером количество пользователей на количество статей), такие что на пересечении строки пользователя и столбца статьи стоит сила их взаимодействия, если взаимодействие было, и стоит ноль, если взаимодействия не было.

In [ ]:
# Ваш код здесь

dtype = pd.SparseDtype(float, fill_value=0)

data_train = pd.pivot_table(
    interactions_train_df,
    values='eventStrength',
    index='personId',
    columns='contentId',
    fill_value=0
)
data_test = pd.pivot_table(
    interactions_test_df,
    values='eventStrength',
    index='personId',
    columns='contentId',
    fill_value=0
)

data_train = pd.DataFrame(
    data_train,
    index=data_train.index | data_test.index,
    columns=data_train.columns | data_test.columns, 
).fillna(0)

data_test = pd.DataFrame(
    data_test,
    index=data_train.index,
    columns=data_train.columns, 
).fillna(0)

data_train, data_test = data_train.astype(dtype).sparse.to_coo(), \
                        data_test.astype(dtype).sparse.to_coo()

print('Shapes:', data_train.shape, data_test.shape)
data_train, data_test

Shapes: (1140, 2984) (1140, 2984)


(<1140x2984 sparse matrix of type '<class 'numpy.float64'>'
 	with 29329 stored elements in COOrdinate format>,
 <1140x2984 sparse matrix of type '<class 'numpy.float64'>'
 	with 9777 stored elements in COOrdinate format>)

## Задание 2. (1 балл)

Обучите модель LightFM с `loss='warp'` и посчитайте *precision@10* на тесте.

In [ ]:
# Ваш код здесь

model = LightFM(loss='warp', random_state=23042021)

model.fit(
    data_train,
    epochs=30,
    num_threads=1,
)

In [ ]:
print('precision@10 на трейне:', precision_at_k(model, data_train, k=10).mean())
print('precision@10 на тесте:', precision_at_k(model, data_test, k=10).mean())

precision@10 на трейне: 0.24163671
precision@10 на тесте: 0.0049898163


## Задание 3. (3 балла)

При вызове метода `fit` LightFM позволяет передавать в `item_features` признаковое описание объектов. Воспользуемся этим. Будем получать признаковое описание из текста статьи в виде [TF-IDF](https://ru.wikipedia.org/wiki/TF-IDF) (можно воспользоваться `TfidfVectorizer` из scikit-learn). Создайте матрицу `feat` размером количесвто статей на размер признакового описание и обучите LightFM с `loss='warp'` и посчитайте precision@10 на тесте.

In [ ]:
# Ваш код здесь
# Склеим текст и заголовок
texts = articles_df[['title', 'text', 'lang']]
texts.index = articles_df['contentId']
texts = texts.reindex(interactions_full_df['contentId'].unique()).fillna('')
texts = texts.apply(lambda x: x['title'] + ' ' + x['text'], axis=1)
texts

contentId
-5065077552540450930    Ranking das maiores seguradoras da Europa - 20...
-6623581327558800021    Spanner, the Google Database That Mastered Tim...
-793729620925729327     Closure Compiler in JavaScript Posted by Sam T...
1469580151036142903     Don't document your code. Code your documentat...
7270966256391553686     Announcing .NET Core 1.0 We are excited to ann...
                                              ...                        
4106497696154898573     Learn Swift Programming Syntax | Udacity Lesso...
-8464215556093549753    HackerRank Women's CodeSprint 2016 Join Women'...
-8202212195240926680    Cover-More to raise $73.3 million to buy Trave...
5518462222339671372     Capturing China's $5 trillion productivity opp...
5937899505996968869     The web is Doom - mobiForge In July 2015 I sug...
Length: 2984, dtype: object

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer()
# Учимся только на статьях из трейна!
vectorizer.fit(texts.reindex(interactions_train_df['contentId'].unique()).fillna(''))

feat = vectorizer.transform(texts)
feat

<2984x65005 sparse matrix of type '<class 'numpy.float64'>'
	with 1038881 stored elements in Compressed Sparse Row format>

In [ ]:
model = LightFM(loss='warp', random_state=23042021)

model.fit(
    data_train,
    item_features=feat,
    epochs=30,
    num_threads=1,
)

In [ ]:
print('precision@10 на трейне:',
      precision_at_k(model, data_train, k=10, item_features=feat).mean()
)
print('precision@10 на тесте:',
      precision_at_k(model, data_test, k=10, item_features=feat).mean()
)

precision@10 на трейне: 0.23929857
precision@10 на тесте: 0.003971487


Стало только хуже -_-

## Задание 4. (2 балла)

В задании 3 мы использовали сырой текст статей. В этом задании необходимо сначала сделать предобработку текста (привести к нижнему регистру, убрать стоп слова, привести слова к номральной форме и т.д.), после чего обучите модель и оценить качество на тестовых данных.

In [ ]:
# Ваш код здесь

# Стоп слова
import nltk
nltk.download("stopwords")
from nltk.corpus import stopwords
en_stopwords = stopwords.words("english")
pt_stopwords = stopwords.words("portuguese")
# Регулярки
import re

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
# Скачаем MyStem
!wget http://download.cdn.yandex.net/mystem/mystem-3.0-linux3.1-64bit.tar.gz &> /dev/null
!tar -xvf mystem-3.0-linux3.1-64bit.tar.gz &> /dev/null
!cp mystem /bin &> /dev/null

from pymystem3 import Mystem
m = Mystem()

In [ ]:
# Как много языков! А мы только с английским и португальским будем работать)
np.unique(articles_df['lang'], return_counts=True)

(array(['en', 'es', 'ja', 'la', 'pt'], dtype=object),
 array([2211,    2,    2,    3,  829]))

In [ ]:
def preprocess(w):
    if not isinstance(w, str) or w is None:
        return ''
    
    # Нижний регистр
    w = w.lower().strip()
    
    # Добавим токен числа
    w = re.sub(r"[0-9]+[.,]*[0-9]*", "num", w)

    # Заменим все пробелами кроме букв и !?
    w = re.sub(r"[^a-zA-Zа-яА-Я?!]+", " ", w)

    # Лемматизируем
    w = [i for i in m.lemmatize(w) if i not in en_stopwords + pt_stopwords]

    w = ''.join(w).strip()

    return w

In [ ]:
texts[1]

'Spanner, the Google Database That Mastered Time, Is Now Open to Everyone About a decade ago, a handful of Google\'s most talented engineers started building a system that seems to defy logic. Called Spanner, it was the first global database, a way of storing information across millions of machines in dozens of data centers spanning multiple continents, and it now underpins everything from Gmail to AdWords, the company\'s primary moneymaker. But it\'s not just the size of this creation that boggles the mind. The real trick is that, even though Spanner stretches across the globe, it behaves as if it\'s in one place. Google can change company data in one part of this database-running an ad, say, or debiting an advertiser\'s account-without contradicting changes made on the other side of the planet. What\'s more, it can readily and reliably replicate data across multiple data centers in multiple parts of the world-and seamlessly retrieve these copies if any one data center goes down. For 

In [ ]:
preprocess(texts[1])

'spanner  google database  mastered time   open  everyone   decade ago  handful  google   talented engineers started building  system  seems  defy logic called spanner    first global database  way  storing information across millions  machines  dozens  data centers spanning multiple continents    underpins everything  gmail  adwords  company  primary moneymaker       size   creation  boggles  mind  real trick   even though spanner stretches across  globe  behaves      one place google  change company data  one part   database running  ad say  debiting  advertiser  account without contradicting changes made    side   planet      readily  reliably replicate data across multiple data centers  multiple parts   world  seamlessly retrieve  copies   one data center goes    truly global business like google  transcontinental consistency  enormously powerful  spanner    seem possible machines   keep databases consistent without constant  heavy communication  communication across  globe took mu

In [ ]:
texts = texts.apply(preprocess)
texts.head()

contentId
-5065077552540450930    ranking  maiores seguradoras  europa  sonho se...
-6623581327558800021    spanner  google database  mastered time   open...
-793729620925729327     closure compiler  javascript posted  sam thoro...
1469580151036142903     document  code code  documentation   one   gre...
7270966256391553686     announcing net core    excited  announce  rele...
dtype: object

In [ ]:
# Снова учим tf-idf
vectorizer = TfidfVectorizer(
    # max_df=0.95, min_df=0.01
)

vectorizer.fit(texts.reindex(interactions_train_df['contentId'].unique()).fillna(''))

feat = vectorizer.transform(texts)
feat

<2984x59585 sparse matrix of type '<class 'numpy.float64'>'
	with 860192 stored elements in Compressed Sparse Row format>

In [ ]:
model = LightFM(loss='warp', random_state=23042021)

model.fit(
    data_train,
    item_features=feat,
    epochs=30,
    num_threads=1,
)

In [ ]:
print('precision@10 на трейне:',
      precision_at_k(model, data_train, k=10, item_features=feat).mean()
)
print('precision@10 на тесте:',
      precision_at_k(model, data_test, k=10, item_features=feat).mean()
)

precision@10 на трейне: 0.24046762
precision@10 на тесте: 0.003971487


Улучшилось ли качество предсказания?

Качество на тренировочной выборке незначительно выросло, а на тестовой выборке не изменилось.

## Задание 5. (2 балла)

Подберите гиперпараметры модели LightFM (`n_components` и др.) для улучшения качества модели.

In [ ]:
# Ваш код здесь
def score_params(data_train, data_test, feat=None, params=None):

    model = LightFM(
        no_components=params['no_components'],
        k=params['k'],
        n=params['n'],
        learning_schedule=params['learning_schedule'],
        loss=params['loss'],
        learning_rate=params['learning_rate'],
        rho=params['rho'],
        epsilon=params['epsilon'],
        item_alpha=params['item_alpha'],
        max_sampled=params['max_sampled'],
        random_state=23042021
    )

    model.fit(
        data_train,
        item_features=feat,
        epochs=50,
        num_threads=1
    )

    print('precision@10 на трейне:',
      precision_at_k(model, data_train, k=10, item_features=feat).mean()
    )
    print('precision@10 на тесте:',
          precision_at_k(model, data_test, k=10, item_features=feat).mean()
    )

    return model, params

In [ ]:
params = {
    'no_components': 5,
    'k': 10,
    'n': 20,
    'learning_schedule': 'adagrad',
    'loss': 'warp', 
    'learning_rate': 0.05,
    'rho': 0.95,
    'epsilon': 1e-06,
    'item_alpha': 0.0,
    'max_sampled': 10,
}

score_params(data_train, data_test, feat, params)

precision@10 на трейне: 0.19046763
precision@10 на тесте: 0.004175153


(<lightfm.lightfm.LightFM at 0x7f4327ec3490>,
 {'epsilon': 1e-06,
  'item_alpha': 0.0,
  'k': 10,
  'learning_rate': 0.05,
  'learning_schedule': 'adagrad',
  'loss': 'warp',
  'max_sampled': 10,
  'n': 20,
  'no_components': 5,
  'rho': 0.95})

In [ ]:
# Переобучились
params = {
    'no_components': 20,
    'k': 10,
    'n': 20,
    'learning_schedule': 'adagrad',
    'loss': 'warp', 
    'learning_rate': 0.05,
    'rho': 0.95,
    'epsilon': 1e-06,
    'item_alpha': 0.0,
    'max_sampled': 20,
}

score_params(data_train, data_test, feat, params)

precision@10 на трейне: 0.40206835
precision@10 на тесте: 0.002851324


(<lightfm.lightfm.LightFM at 0x7f4327fa3ed0>,
 {'epsilon': 1e-06,
  'item_alpha': 0.0,
  'k': 10,
  'learning_rate': 0.05,
  'learning_schedule': 'adagrad',
  'loss': 'warp',
  'max_sampled': 20,
  'n': 20,
  'no_components': 20,
  'rho': 0.95})

In [ ]:
params = {
    'no_components': 6,
    'k': 5,
    'n': 10,
    'learning_schedule': 'adagrad',
    'loss': 'warp', 
    'learning_rate': 0.05,
    'rho': 0.95,
    'epsilon': 1e-06,
    'item_alpha': 0.0,
    'max_sampled': 11,
}

score_params(data_train, data_test, None, params)

precision@10 на трейне: 0.19991007
precision@10 на тесте: 0.0056008147


(<lightfm.lightfm.LightFM at 0x7f4327fa3bd0>,
 {'epsilon': 1e-06,
  'item_alpha': 0.0,
  'k': 5,
  'learning_rate': 0.05,
  'learning_schedule': 'adagrad',
  'loss': 'warp',
  'max_sampled': 11,
  'n': 10,
  'no_components': 6,
  'rho': 0.95})

0.0056 - максимум, который я смог добиться на тесте :(

## Бонусное задание. (3 балла)

Выше мы использовали достаточно простое представление текста статьи в виде TF-IDF. В этом задании Вам нужно представить текст статьи (можно вместе с заголовком) в виде эмбеддинга полученного с помощью рекуррентной сети или трансформера (можно использовать любую предобученную модель, которая Вам нравится). Обучите модель с ипользованием этих эмеддингов и сравните результаты с предыдущими.

In [ ]:
# Ваш код здесь
!pip install transformers

     |████████████████████████████████| 2.1MB 8.1MB/s 
     |████████████████████████████████| 3.3MB 36.1MB/s 
     |████████████████████████████████| 901kB 55.3MB/s 


In [ ]:
from transformers import BertTokenizer, BertModel
import torch
# Прогресс бар
from tqdm.notebook import tqdm

tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-uncased')
bert = BertModel.from_pretrained('bert-base-multilingual-uncased').cuda().eval()

In [ ]:
# Склеим текст и заголовок как и в tf-idf
texts = articles_df[['title', 'text', 'lang']]
texts.index = articles_df['contentId']
texts = texts.reindex(interactions_full_df['contentId'].unique()).fillna('')
texts = texts.apply(lambda x: x['title'] + ' ' + tokenizer.sep_token + ' ' + x['text'], axis=1)
texts

contentId
-5065077552540450930    Ranking das maiores seguradoras da Europa - 20...
-6623581327558800021    Spanner, the Google Database That Mastered Tim...
-793729620925729327     Closure Compiler in JavaScript [SEP] Posted by...
1469580151036142903     Don't document your code. Code your documentat...
7270966256391553686     Announcing .NET Core 1.0 [SEP] We are excited ...
                                              ...                        
4106497696154898573     Learn Swift Programming Syntax | Udacity [SEP]...
-8464215556093549753    HackerRank Women's CodeSprint 2016 [SEP] Join ...
-8202212195240926680    Cover-More to raise $73.3 million to buy Trave...
5518462222339671372     Capturing China's $5 trillion productivity opp...
5937899505996968869     The web is Doom - mobiForge [SEP] In July 2015...
Length: 2984, dtype: object

In [ ]:
# Функция для токенизации всех текстов
def tokenize(texts, tokenizer, batch_size=16):
    batched_inputs = []
    for batch in tqdm(np.array_split(texts, len(texts) // batch_size + 1)):
        tokens = tokenizer(
                batch.tolist(),
                return_tensors="pt",
                truncation=True,
                padding=True
        ) # Будем считать ембединги только по первым 512 токенам (максимум в модели)
        tokens = {k: v.cuda() for k, v in tokens.items()}
        batched_inputs.append(tokens)
    return batched_inputs
 
# Функция для создания эмбедингов текстов (средний эмбединг статьи)
def make_embds(texts, bert, tokenizer, batch_size=16, strategy='mean'):
    feat = tokenize(texts, tokenizer, batch_size)
    res = []
    with torch.no_grad(): # Обязательно выключить градиенты, иначе память убивается
        for batch in tqdm(feat):
            if strategy == 'mean':
                res.append(bert(**batch).last_hidden_state.mean(dim=1).cpu())
            elif strategy == 'pooler':
                res.append(bert(**batch).pooler_output.cpu())
            else: 
                raise ValueError()
    res = torch.cat(res)
    return res

In [ ]:
# Делаем эмбединги
embds = make_embds(texts, bert, tokenizer, strategy='mean')
embds.shape

torch.Size([2984, 768])

In [ ]:
# Делаем эмбединги (pooler)
embds_p = make_embds(texts, bert, tokenizer, strategy='pooler')
embds_p.shape

torch.Size([2984, 768])

In [ ]:
from sklearn.decomposition import PCA

pca = PCA(n_components=100)
feat_pca = pca.fit_transform(embds_p.numpy())
feat_pca.shape

(2984, 100)

In [ ]:
from scipy.sparse import coo_matrix

feat = coo_matrix(embds.numpy())
feat_pca = coo_matrix(feat_pca)
feat, feat_pca

(<2984x768 sparse matrix of type '<class 'numpy.float32'>'
 	with 2291712 stored elements in COOrdinate format>,
 <2984x100 sparse matrix of type '<class 'numpy.float32'>'
 	with 298400 stored elements in COOrdinate format>)

In [ ]:
params = {
    'no_components': 25,
    'k': 5,
    'n': 10,
    'learning_schedule': 'adagrad',
    'loss': 'warp', 
    'learning_rate': 0.05,
    'rho': 0.95,
    'epsilon': 1e-06,
    'item_alpha': 0.0,
    'max_sampled': 11,
}

score_params(data_train, data_test, feat, params)

precision@10 на трейне: 0.14523381
precision@10 на тесте: 0.0018329939


(<lightfm.lightfm.LightFM at 0x7fee00424890>,
 {'epsilon': 1e-06,
  'item_alpha': 0.0,
  'k': 5,
  'learning_rate': 0.05,
  'learning_schedule': 'adagrad',
  'loss': 'warp',
  'max_sampled': 11,
  'n': 10,
  'no_components': 25,
  'rho': 0.95})

In [ ]:
model = LightFM(loss='warp', random_state=23042021)

model.fit(
    data_train,
    item_features=feat,
    epochs=50,
    num_threads=1,
)

print('precision@10 на трейне:',
      precision_at_k(model, data_train, k=10, item_features=feat).mean()
)
print('precision@10 на тесте:',
      precision_at_k(model, data_test, k=10, item_features=feat).mean()
)

precision@10 на трейне: 0.098471224
precision@10 на тесте: 0.0018329938


In [ ]:
# Посмотрим качество, если применить PCA к эмбеддингам
model = LightFM(loss='warp',
                no_components=100,
                random_state=23042021)

model.fit(
    data_train,
    item_features=feat_pca,
    epochs=50,
    num_threads=1,
)

print('precision@10 на трейне:',
      precision_at_k(model, data_train, k=10, item_features=feat_pca).mean()
)
print('precision@10 на тесте:',
      precision_at_k(model, data_test, k=10, item_features=feat_pca).mean()
)

precision@10 на трейне: 0.11258994
precision@10 на тесте: 0.0032586558


С эмбедингами из BERT'a получилось еще хуже. Данный результат также не зависит от выбора агрегирования векторных представлений токенов. Как среднее по всем эмбедингам, так и CLS-эмбединг дают одинаковые результаты. Либо 50 эпох мало, либо нужно дообучать представления с BERT'a